In [2]:
#Used Libraries
import pandas as pd
from sqlalchemy import create_engine
import psycopg2


In [2]:
#Check for duplicates in rep_sales table:

queryResult("""
SELECT "Item","OrderDate","Rep","Region", COUNT(*)
FROM rep_sales
GROUP BY
    "Item","OrderDate","Rep","Region"
HAVING COUNT(*) > 1
""")

,Item,OrderDate,Rep,Region,count


no duplicates

In [3]:
#Check for duplicates in item table:
queryResult("""
SELECT "Item","MinPrice", COUNT(*)
FROM item
GROUP BY
    "Item","MinPrice"
HAVING COUNT(*) > 1
""")

,Item,MinPrice,count
0,Binder,1.99,2
1,Pen,1.75,2
2,Desk,125.00,2
3,Pen Set,4.99,2
4,Pencil,1.29,2
5,Marker Set,5.99,2


In [4]:
#1. What is the total income in the data?
queryResult("""
SELECT
    COUNT(DISTINCT("Item")),
    SUM("Total")
FROM
    rep_sales
""")

,count,sum
0,6,28867.97


In [5]:
#2. Which sales rep brought the most income?

queryResult("""
SELECT
    DISTINCT("Rep"),
    SUM("Units") AS number_of_units,
    SUM("Total") AS income
FROM
    rep_sales
GROUP BY
    "Rep"
ORDER BY
    income DESC
LIMIT 1;
""")

,Rep,number_of_units,income
0,Kivell,240,3554.23


In [6]:
#3. Which item brought the most income?

queryResult("""
SELECT
    DISTINCT("Item"),
    SUM("Units") AS number_of_units,
    SUM("Total") AS income
FROM
    rep_sales
GROUP BY
    "Item"
ORDER BY
    income DESC
LIMIT 1;
""")

,Item,number_of_units,income
0,Binder,708,8229.12


In [7]:
#4. Which region sold the most desks?

queryResult("""
SELECT
    DISTINCT("Region"),
    "Item",
    SUM("Units") AS number_of_units
FROM
    rep_sales
WHERE
    "Item" = 'Desk'
GROUP BY
    "Region",
    "Item"
ORDER BY
    number_of_units DESC
LIMIT 1;
""")

,Region,Item,number_of_units
0,West,Desk,9


In [8]:
#5. Which sales rep sold the highest average price a piece for the item pen?

queryResult("""
SELECT
    "Rep",
    "Item",
    AVG("Unit_Cost") AS avg_price
FROM
    rep_sales
WHERE
    "Item" = 'Pen'
GROUP BY
    "Rep",
    "Item"
ORDER BY
    avg_price DESC
LIMIT 1;
""")

,Rep,Item,avg_price
0,Parent,Pen,19.99


In [3]:
#6. Which sales rep sold the most units between April – September of 2020?

queryResult("""
SELECT
    DISTINCT("Rep"),
    SUM("Units") AS number_of_units
FROM
    rep_sales
WHERE
    EXTRACT(MONTH FROM "OrderDate" :: date) >= 4 AND EXTRACT(MONTH FROM "OrderDate" :: date) <=9 AND
    EXTRACT(YEAR FROM "OrderDate" :: date) = '2020'
GROUP BY
    "Rep"
ORDER BY
    number_of_units DESC
LIMIT 1;
""")

,Rep,number_of_units
0,Andrews,309


In [15]:
#7. Which sales rep had the highest price difference from the minimum price on the item pen set?

queryResult("""
WITH 
    item_new as (
    SELECT
        DISTINCT("Item"),
        "MinPrice"
    FROM item) 
SELECT
    rep_sales."Rep",
    rep_sales."Item",
    item_new."MinPrice",
    rep_sales."Unit_Cost",
    (rep_sales."Unit_Cost" - item_new."MinPrice") AS diff
FROM
    rep_sales LEFT JOIN item_new ON rep_sales."Item" = item_new."Item"
WHERE
    rep_sales."Item" = 'Pen Set'
ORDER BY
    diff DESC
LIMIT 31;
""")

,Rep,Item,MinPrice,Unit_Cost,diff
0,Kivell,Pen Set,4.99,17.00,12.01
1,Jones,Pen Set,4.99,15.99,11.00
2,Parent,Pen Set,4.99,12.99,8.00
3,Morgan,Pen Set,4.99,12.49,7.50
4,Thompson,Pen Set,4.99,8.99,4.00
5,Morgan,Pen Set,4.99,7.25,2.26
6,Jones,Pen Set,4.99,4.99,0.00
7,Jardine,Pen Set,4.99,4.99,0.00
8,Kivell,Pen Set,4.99,4.99,0.00


In [96]:
#8. Was there a sales rep that sold an item below the minimum price? If so, which item was it and what was the percent difference from the minimum price?

queryResult("""
WITH 
    item_new as (
    SELECT
        DISTINCT("Item"),
        "MinPrice"
    FROM item) 
SELECT
    rep_sales."Rep",
    rep_sales."Item",
    item_new."MinPrice",
    rep_sales."Unit_Cost" AS sale_price,
    (rep_sales."Unit_Cost" - item_new."MinPrice")/item_new."MinPrice" AS diff
FROM
    rep_sales LEFT JOIN item_new ON rep_sales."Item" = item_new."Item"
WHERE
    rep_sales."Unit_Cost" < item_new."MinPrice"
ORDER BY
    diff
LIMIT 5;
""")

,Rep,Item,MinPrice,sale_price,diff
0,Gill,Pen,1.75,1.49,-0.148571
1,Andrews,Pen,1.75,1.50,-0.142857
2,Thompson,Pen,1.75,1.59,-0.091429


1. Thompson sold the item Pen 9% below it's min price
2. Andrew sold the item Pen 14.2% below it's min price
3. Gill sold the item Pen 14.8% below it's min price

In [55]:
#9. Which region had the highest share of price difference ?
#(where percent price difference is the amount above min price divided by the total amount from that region)

queryResult("""
WITH 
    item_new as (
    SELECT
        DISTINCT("Item"),
        "MinPrice"
    FROM item), 
rep_sales_new as ( 
    SELECT
        rep_sales."Region" AS region,
        SUM(rep_sales."Unit_Cost" - item_new."MinPrice") AS diff
    FROM
        rep_sales LEFT JOIN item_new ON rep_sales."Item" = item_new."Item"
    WHERE
        rep_sales."Unit_Cost" > item_new."MinPrice"
    GROUP BY
        rep_sales."Region"),
    total as (
        SELECT
        COUNT(rep_sales_new.region),
        SUM(rep_sales_new.diff) AS total_diff
    FROM
        rep_sales_new)
SELECT
    rep_sales_new.region,
    rep_sales_new.diff, 
    (rep_sales_new.diff/ total.total_diff) AS share
FROM 
    rep_sales_new, total
""")

,region,diff,share
0,West,298.61,0.407475
1,East,288.16,0.393215
2,Central,146.06,0.199310


In [13]:
#10. For each sale above the minimum price the rep gets 10% commission of the total price as a bonus. Which 3 sales reps have earned the most commission (show their commission as well)?

queryResult("""
WITH 
    item_new as (
    SELECT
        DISTINCT("Item"),
        "MinPrice"
    FROM item) 
SELECT
    rep_sales."Rep",
    SUM(rep_sales."Total" * 0.1) AS comission
FROM
    rep_sales LEFT JOIN item_new ON rep_sales."Item" = item_new."Item"
WHERE 
    rep_sales."Unit_Cost" > item_new."MinPrice"
GROUP BY
    rep_sales."Rep"
ORDER BY
    comission DESC
LIMIT 3;
""")

,Rep,comission
0,Jardine,275.730
1,Thompson,272.633
2,Morgan,266.998
